# Installing Relevant Libraries


In [ ]:
!pip install gdown #to download the dataset from google drive
!pip install -U langchain-community # to work with the langchain llm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00


### Importing Relevant Libraries

In [ ]:
import gdown # to download dataset from google drive
import pandas as pd #to import dataset as
from collections import defaultdict # to store the context map
from langchain.llms import HuggingFaceHub # to initialize the LLM

# (Down)Loading the dataset from google drive:

For reference, This is link to dataset uploaded on google drive:

https://drive.google.com/drive/folders/1gvZEHd_Qr60flkoxfOEjJ3G956w-4eSH?usp=sharing

and we are using v1.1 and specifically, .csv files (converted from .json files)

In [ ]:
validation_dataset_url = 'https://drive.google.com/uc?id=1bGNCDoItbvFbBWoDgBF8YcLFI1Te5RmV'
training_dataset_url = 'https://drive.google.com/uc?id=1-vsbRTLKgo2PalcG3AargcBblytlGQd1'

validation_dataset_filename = 'validation_dataset.csv'
training_dataset_filename = 'training_dataset.csv'

gdown.download(validation_dataset_url, validation_dataset_filename, quiet=True)
gdown.download(training_dataset_url, training_dataset_filename, quiet=True)


'training_dataset.csv'

### Importing Dataset as DF

In [ ]:
df = pd.read_csv(training_dataset_filename)

print(df.head())

                      index  \
0  5733be284776f41900661182   
1  5733be284776f4190066117f   
2  5733be284776f41900661180   
3  5733be284776f41900661181   
4  5733be284776f4190066117e   

                                            question  \
0  To whom did the Virgin Mary allegedly appear i...   
1  What is in front of the Notre Dame Main Building?   
2  The Basilica of the Sacred heart at Notre Dame...   
3                  What is the Grotto at Notre Dame?   
4  What sits on top of the Main Building at Notre...   

                                             context  answer_start  \
0  Architecturally, the school has a Catholic cha...           515   
1  Architecturally, the school has a Catholic cha...           188   
2  Architecturally, the school has a Catholic cha...           279   
3  Architecturally, the school has a Catholic cha...           381   
4  Architecturally, the school has a Catholic cha...            92   

                                      text  c_id  
0   

### Making a hashmap of the context.

kind of a pre-processing step, so it will be easier to retrive contexts and questions on demand.

In [ ]:
context_map = defaultdict(list)

for _, row in df.iterrows():
    context = row['context']
    question = row['question']
    answer = row['text']
    context_map[context].append((question, answer))

for context, qa_pairs in list(context_map.items())[:1]:
    print("Context:", context[:200], "...")
    print("QA Pairs:")
    for q, a in qa_pairs:
        print(f"Q: {q}")
        print(f"A: {a}")
        print()

# context: [(q1,a2) , (q2,a2)]   context2: [(q1,a1)]



Context: Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper sta ...
QA Pairs:
Q: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
A: Saint Bernadette Soubirous

Q: What is in front of the Notre Dame Main Building?
A: a copper statue of Christ

Q: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
A: the Main Building

Q: What is the Grotto at Notre Dame?
A: a Marian place of prayer and reflection

Q: What sits on top of the Main Building at Notre Dame?
A: a golden statue of the Virgin Mary



### get_context_questions

A helper function to get as many chunks and questions we want to test limits/find out optimal context input length of the specific LLM we are testing with.

In [ ]:
def get_context_questions(k=2, q=4):
    contexts = []
    questions = []
    selected_contexts = list(context_map.items())[:k]
    for context, qa_pairs in selected_contexts:
        contexts.append(context)
        selected_qa = qa_pairs[:q]
        for question, _ in selected_qa:
            questions.append(question)
    return contexts, questions

#testing:
contexts, questions = get_context_questions(k=2, q=4)

print("Contexts:")
for c in contexts:
    print(c[:200], "...")

print("\nQuestions:")
for q in questions:
    print(q)



Contexts:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper sta ...
As at most other universities, Notre Dame's students run a number of news media outlets. The nine student-run outlets include three newspapers, both a radio and television station, and several magazin ...

Questions:
To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
What is in front of the Notre Dame Main Building?
The Basilica of the Sacred heart at Notre Dame is beside to which structure?
What is the Grotto at Notre Dame?
When did the Scholastic Magazine of Notre dame begin publishing?
How often is Notre Dame's the Juggler published?
What is the daily student paper at Notre Dame called?
How many student news papers are found at Notre Dame?


In [ ]:
# Agent class taken from assignment 3
class Agent:
    def __init__(self, context_map, model_name="mistralai/Mistral-7B-Instruct-v0.3", access_token="TOKEN-HERE"):
        self.context_map = context_map
        self.llm = HuggingFaceHub(
            repo_id=model_name,
            model_kwargs={"temperature": 0.3, "max_length": 200},
            huggingfacehub_api_token=access_token
        )

    def generate_response(self, prompt):
        return self.llm(prompt)

    def get_context_questions(self, k=2, q=4):
        contexts = []
        questions_answers = []
        selected_contexts = list(self.context_map.items())[:k]
        for context, qa_pairs in selected_contexts:
            contexts.append(context)
            selected_qa = qa_pairs[:q]
            questions_answers.append(selected_qa)
        return contexts, questions_answers

    #Fixed issue of seperate reponses for each context - Affan
    def ask_questions(self, k=2, q=4):
        contexts, questions_answers_by_context = self.get_context_questions(k, q)

        # Combine all contexts into one
        combined_context = "\n\n".join(contexts)

        # Combine all question-answer pairs
        all_questions_answers = []
        for qa_pairs in questions_answers_by_context:
            all_questions_answers.extend(qa_pairs)

        # Prepare the combined prompt
        prompt_lines = [
            "answer these questions:",
            "Answer as precisely as you can, in as minimum words as you can",
            "Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer"
        ]

        # Add all questions to the prompt
        for i, (question, _) in enumerate(all_questions_answers, 1):
            prompt_lines.append(f"Q{i}: {question}")

        prompt_lines.append("\nfrom this context:")
        prompt_lines.append(combined_context)

        prompt = "\n".join(prompt_lines)

        # Calculate and print prompt length
        char_length = len(prompt)
        # Simple token estimation (not precise but gives a rough idea)
        token_count = len(prompt.split())

        print(f"\n🧠 Combined Prompt:")
        print(f"Length: {char_length} characters, approximately {token_count} tokens")
        print(f"{prompt}\n")

        # Get response from LLM
        response = self.generate_response(prompt)

        print(f"📤 LLM Response:\n{response}\n{'='*60}")

        # Improved parsing of LLM response - handle different response formats
        model_answers = {}

        # Try to parse answers with A1: format first
        for line in response.strip().split("\n"):
            line = line.strip()
            if line.startswith("A") and ":" in line:
                try:
                    key, value = line.split(":", 1)
                    if key.strip()[1:].isdigit():  # Check if A1, A2, etc.
                        model_answers[key.strip()] = value.strip()
                except:
                    pass

        # If we couldn't find any answers in A1: format, try to extract answers from the full response
        if not model_answers:
            # Look for answers that begin with Q1, Q2, etc. followed by the answer
            lines = response.strip().split("\n")
            i = 0
            while i < len(lines):
                if lines[i].strip().startswith("Q") and ":" in lines[i]:
                    q_num = lines[i].split(":")[0].strip()[1:]
                    if q_num.isdigit() and i + 1 < len(lines):
                        # Assume the next line or next non-Q line is the answer
                        next_i = i + 1
                        while next_i < len(lines) and not lines[next_i].strip().startswith("Q"):
                            next_i += 1
                        answer = "\n".join(lines[i+1:next_i]).strip()
                        model_answers[f"A{q_num}"] = answer
                        i = next_i - 1
                i += 1

        # If we still don't have answers, try to extract in order from the response
        if not model_answers:
            lines = [line.strip() for line in response.strip().split("\n") if line.strip()]
            if len(lines) >= len(all_questions_answers):
                for i in range(len(all_questions_answers)):
                    model_answers[f"A{i+1}"] = lines[i]

        # Side-by-side comparison
        print(f"\n🔍 Side-by-Side Comparison:\n")
        for i, (question, ground_truth) in enumerate(all_questions_answers, 1):
            key = f"A{i}"
            llm_answer = model_answers.get(key, "[No Answer Returned]")
            print(f"Q{i}: {question}")
            print(f"✅ Ground Truth: {ground_truth}")
            print(f"🤖 LLM Answer  : {llm_answer}")
            print("-" * 60)

agent = Agent(context_map)
# agent.ask_questions(k=1, q=5)



<ipython-input-7-49010b5682ff>:5: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  self.llm = HuggingFaceHub(


## Testing

Now coming to the testing part, we will vary k and q and will make comments and observations for each experiment. based on those observations, we will plan our approach.

We will use humans as evaluators to check if the LLM has answered correctly or not, we could also have used LLMs as evaluators but they also come with their own problems, and we dont want that to be a variable in our research, specially when we dont have access to SOTA LLMs like ChatGPT 4o etc

### K1 Q3

Observation: All Questions correctly answered, question 3 was previously left in the middle, now we got answers for that as well.

In [ ]:
# agent.ask_questions(k=1, q=3)


🧠 Combined Prompt:
Length: 1089 characters, approximately 196 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?

from this context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soub

<ipython-input-7-49010b5682ff>:12: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  return self.llm(prompt)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), se

📤 LLM Response:
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?

from this context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a 

### K2 Q4

In [ ]:
# agent.ask_questions(k=2, q=4)


🧠 Combined Prompt:
Length: 2772 characters, approximately 462 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: When did the Scholastic Magazine of Notre dame begin publishing?
Q6: How often is Notre Dame's the Juggler published?
Q7: What is the daily student paper at Notre Dame called?
Q8: How many student news papers are found at Notre Dame?

from this context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


📤 LLM Response:
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: When did the Scholastic Magazine of Notre dame begin publishing?
Q6: How often is Notre Dame's the Juggler published?
Q7: What is the daily student paper at Notre Dame called?
Q8: How many student news papers are found at Notre Dame?

from this context:
Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica o

### K4 Q5

In [ ]:
# agent.ask_questions(k=4, q=5)


🧠 Combined Prompt:
Length: 5105 characters, approximately 830 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy C

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


📤 LLM Response:
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy Cross?
Q13: What is the oldest structure at Notre Dame?

# K6 Q5

In [ ]:
agent.ask_questions(k=6, q=5)


🧠 Combined Prompt:
Length: 7893 characters, approximately 1282 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3 (Request ID: Root=1-6804a8a0-66a762840899790b066c03e5;b04c55b2-2513-4b24-b6b6-a7877308f96b)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

### K8 Q5

In [ ]:
# agent.ask_questions(k=8, q=5)


🧠 Combined Prompt:
Length: 9693 characters, approximately 1587 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy 

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


📤 LLM Response:
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy Cross?
Q13: What is the oldest structure at Notre Dame?

### K16 Q5

In [ ]:
# agent.ask_questions(k=16, q=5)


🧠 Combined Prompt:
Length: 20766 characters, approximately 3356 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


📤 LLM Response:
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy Cross?
Q13: What is the oldest structure at Notre Dame?

### K24 Q5

In [ ]:
# agent.ask_questions(k=24, q=5)

### K32 Q5

In [ ]:
# agent.ask_questions(k=32, q=5)


🧠 Combined Prompt:
Length: 42962 characters, approximately 7000 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Holy

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3

### K64 Q5

In [ ]:
# agent.ask_questions(k=64, q=5)


🧠 Combined Prompt:
Length: 82656 characters, approximately 13535 tokens
answer these questions:
Answer as precisely as you can, in as minimum words as you can
Format your answers with 'A1:', 'A2:', etc. at the beginning of each answer
Q1: To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
Q2: What is in front of the Notre Dame Main Building?
Q3: The Basilica of the Sacred heart at Notre Dame is beside to which structure?
Q4: What is the Grotto at Notre Dame?
Q5: What sits on top of the Main Building at Notre Dame?
Q6: When did the Scholastic Magazine of Notre dame begin publishing?
Q7: How often is Notre Dame's the Juggler published?
Q8: What is the daily student paper at Notre Dame called?
Q9: How many student news papers are found at Notre Dame?
Q10: In what year did the student paper Common Sense begin publication at Notre Dame?
Q11: Where is the headquarters of the Congregation of the Holy Cross?
Q12: What is the primary seminary of the Congregation of the Hol

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


HfHubHTTPError: 504 Server Error: Gateway Time-out for url: https://router.huggingface.co/hf-inference/models/mistralai/Mistral-7B-Instruct-v0.3